In [1]:
from influxdb.commands import InfluxDBQueries, DataProcessor, LSTMModel
import warnings
import pandas as pd
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [2]:
df_train = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2024-01-01T01:00:00Z",
                                                stop="2025-01-01T01:00:00Z")
df_val = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2025-01-01T01:00:00Z",
                                              stop="2025-03-01T01:00:00Z")
df_test = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2025-03-01T01:00:00Z",
                                               stop="2025-04-20T01:00:00Z")

In [3]:
df_combined = pd.concat([df_train, df_test, df_val])

In [4]:
df_combined, scaler, columns, sclaer_y = DataProcessor.add_features(df_combined, classification=True, group_time='5m')

In [5]:
df_combined.dropna(inplace=True)

In [6]:
columns, boruta = DataProcessor.select_features_boruta(df_combined, columns, classification=True)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	13
Tentative: 	10
Rejected: 	19
Iteration: 	9 / 100
Confirmed: 	13
Tentative: 	10
Rejected: 	19
Iteration: 	10 / 100
Confirmed: 	13
Tentative: 	10
Rejected: 	19
Iteration: 	11 / 100
Confirmed: 	13
Tentative: 	10
Rejected: 	19
Iteration: 	12 / 100
Confirmed: 	14
Tentative: 	9
Rejected: 	19
Iteration: 	13 / 100
Confirmed: 	14
Tentative: 	8
Rejected: 	20
Iteration: 	14 / 100
Confirmed: 	14
Tentative: 	8
Rejected: 	20
Iteration: 	15 / 100
Confirmed: 	14
Tentative: 	8
Rejected: 	20
Iteration: 	16 / 100
Confirmed: 	14
Tentative: 	7
Re

In [7]:
df_train = df_combined.loc[df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') < df_val.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')]
df_val = df_combined.loc[
    (df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') >= df_val.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')) &
    (df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') < df_test.index[0].strftime('%Y-%m-%dT%H:%M:%SZ'))]
df_test = df_combined.loc[
    df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') >= df_test.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')]

In [8]:
df_train = df_train.dropna()
df_test = df_test.dropna()
df_val = df_val.dropna()

In [9]:
X_train, y_train = DataProcessor.make_sequences(df_train, columns_to_select=columns, window_size=288,forecast_horizon=1, classification=True)

In [10]:
X_val, y_val = DataProcessor.make_sequences(df_val, columns_to_select=columns, window_size=288, forecast_horizon=1, classification=True)

In [11]:
lstm_forecaster = LSTMModel(input_shape=(288, len(columns)), output_steps=1, classification=True)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 288, 64)        │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 288, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 288, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 288, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 288, 128)       │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 288, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 288, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 80)             │        66,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 80)             │           320 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 170,001 (664.07 KB)

 Trainable params: 169,457 (661.94 KB)

 Non-trainable params: 544 (2.12 KB)

In [12]:
lstm_forecaster.fit(X_train, y_train, X_val, y_val, epochs=300, batch_size=128, patience=20)

Training started with EarlyStopping (patience=20)...
Epoch 1/300
818/818 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5021 - auc: 0.5009 - loss: 0.8060
Epoch 1: val_loss improved from inf to 0.69265, saving model to models/best_model.keras
818/818 ━━━━━━━━━━━━━━━━━━━━ 1030s 1s/step - accuracy: 0.5021 - auc: 0.5009 - loss: 0.8059 - val_accuracy: 0.5153 - val_auc: 0.5197 - val_loss: 0.6926 - learning_rate: 0.0010
Epoch 2/300
818/818 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5046 - auc: 0.5080 - loss: 0.6942
Epoch 2: val_loss did not improve from 0.69265
818/818 ━━━━━━━━━━━━━━━━━━━━ 1039s 1s/step - accuracy: 0.5046 - auc: 0.5080 - loss: 0.6942 - val_accuracy: 0.5037 - val_auc: 0.5084 - val_loss: 0.6942 - learning_rate: 0.0010
Epoch 3/300
818/818 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5066 - auc: 0.5110 - loss: 0.6936
Epoch 3: val_loss did not improve from 0.69265
818/818 ━━━━━━━━━━━━━━━━━━━━ 1038s 1s/step - accuracy: 0.5066 - auc: 0.5110 - loss: 0.6936 - val_accuracy: 0.4980 -

In [13]:
X_test, y_test = DataProcessor.make_sequences(df_test, columns_to_select=columns, window_size=288, forecast_horizon=1, classification=True)

In [14]:
lstm_forecaster.evaluate(X_test, y_test)

441/441 ━━━━━━━━━━━━━━━━━━━━ 41s 94ms/step - accuracy: 0.5192 - auc: 0.5231 - loss: 0.6924
Test Loss: 0.6920283436775208, MSE: 0.521896243095398, Directional Accuracy: <function directional_accuracy at 0x145bee7a0>
441/441 ━━━━━━━━━━━━━━━━━━━━ 41s 92ms/step
Confidence-Filtered Accuracy: 0.50, Coverage: 100.00%


0.6920283436775208

In [15]:
lstm_forecaster.save_model()

Model saved successfully to models/20250430-170635.keras
